In [1]:
pip install selenium beautifulsoup4 openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 9.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
from random import uniform

# WebDriver 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # Headless 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

def get_keyword(text):
    return text.replace(" ", "%20")

def sort_kind(index):
    if index == 1:
        return 'vcount'
    elif index == 2:
        return 'date'
    else:
        return 'none'

keyword = '강아지 기침'  # 크롤링할 키워드 설정
driver.get('https://kin.naver.com/search/list.nhn?query=' + get_keyword(keyword))
time.sleep(uniform(0.1, 1.0))

page_index = 1
f = '2020.07.11'  # 크롤링 시작 일자
t = '2024.07.11'  # 크롤링 종료 일자
period_txt = "&period=" + f + ".%7C" + t + "."
_sort_kind = sort_kind(2)

page_url = []

# 최대 페이지 수 설정
max_pages = 1

while page_index <= max_pages:
    time.sleep(uniform(0.1, 1.0))
    driver.get('https://kin.naver.com/search/list.nhn?' + "&sort=" + _sort_kind + '&query=' + get_keyword(keyword) + period_txt + "&section=kin" + "&page=" + str(page_index))
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    tags = soup.find_all('a', class_="_searchListTitleAnchor")
    for tag in tags:
        url = tag['href']
        page_url.append(url)

    page_index += 1  # 페이지 증가

data = []

for url in page_url:
    driver.get(url)
    time.sleep(uniform(0.5, 1.0))  # 페이지 로딩 시간을 위한 대기
    try:
        title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[1]'))
        ).text
    except:
        title = ""
    
    # "강아지기침" 키워드가 제목에 포함되어 있는지 확인
    if keyword in title:
        try:
            question_txt = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[3]'))
            ).text
        except:
            question_txt = ""
        
        answers = driver.find_elements(By.CSS_SELECTOR, ".se-main-container .se-component-content")
        answer_text = [answer.text for answer in answers]
        
        if not answer_text:  # 답변이 없는 경우도 처리
            answer_text = [""]

        for answer in answer_text:
            data.append([title, question_txt, answer])

dog_df = pd.DataFrame(data, columns=['제목', '질문', '답변'])
dog_df

# 필요에 따라 데이터를 파일로 저장
# df.to_excel('naver_kin_crawling_result.xlsx', index=False)

driver.quit()


In [17]:
dog_df.to_excel('naver_kin_crawling_result.xlsx', index=False)

In [3]:

# '제목' 열에서 '질문' 단어 제거
dog_df['제목'] = dog_df['제목'].str.replace('질문', '')

# '제목' 열에서 '질문' 단어 제거
dog_df['제목'] = dog_df['제목'].str.replace('\n', '')
dog_df

,제목,질문,답변
0,강아지 기침소리좀 봐주세오 ㅠㅠ,요즘 들어 부쩍 기침이 잦아서요 거위소리도 내고 영상처럼 색색거리는? 기침도 하고 ...,안녕하세요\n수의사 이재민입니다.\n기침때문에 걱정되시겠어요\n청색증이 맞네요..심...
1,강아지 기침 소리,울애긴데요 ..\n8살이에요\n기침소리 이유가 있나요?\n요리할때 주방와서 더 심한...,울애긴데요 ..\n8살이에요\n기침소리 이유가 있나요?\n요리할때 주방와서 더 심한...
2,강아지 기침 경련,8살 남자 포메인데요\n오늘 갑자기 숨이 뒤로 넘어가는 듯한 기침을 하길래 동네 병...,"안녕하세요. ""동물의료상담 지식인""에 선정된 부천 중동 ""하모니동물병원"" 원장입니다..."


In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from random import uniform

# WebDriver 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # Headless 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

def get_keyword(text):
    return text.replace(" ", "%20")

def sort_kind(index):
    if index == 1:
        return 'vcount'
    elif index == 2:
        return 'date'
    else:
        return 'none'

keyword = '강아지'  # 크롤링할 키워드 설정
driver.get('https://kin.naver.com/search/list.nhn?query=' + get_keyword(keyword))
time.sleep(uniform(0.1, 1.0))

page_index = 1
f = '2020.06.10'  # 크롤링 시작 일자
t = '2024.07.11'  # 크롤링 종료 일자
period_txt = "&period=" + f + ".%7C" + t + "."
_sort_kind = sort_kind(2)

page_url = []

# 최대 페이지 수 설정
max_pages = 1

while page_index <= max_pages:
    time.sleep(uniform(0.1, 1.0))
    driver.get('https://kin.naver.com/search/list.nhn?' + "&sort=" + _sort_kind + '&query=' + get_keyword(keyword) + period_txt + "&section=kin" + "&page=" + str(page_index))
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    tags = soup.find_all('a', class_="_searchListTitleAnchor")
    for tag in tags:
        url = tag['href']
        page_url.append(url)

    page_index += 1  # 페이지 증가

data = []

for url in page_url:
    driver.get(url)
    time.sleep(uniform(0.5, 1.0))  # 페이지 로딩 시간을 위한 대기
    try:
        title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[1]'))
        ).text
    except:
        title = ""
    
    # "강아지기침" 키워드가 제목에 포함되어 있는지 확인
    if keyword in title:
        try:
            question_txt = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[3]'))
            ).text
        except:
            question_txt = ""
        try:
            date = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'span.blind'))
            ).text
        except:
            date = ""
        
        answers = driver.find_elements(By.CSS_SELECTOR, ".se-main-container .se-component-content")
        answer_text = [answer.text for answer in answers]
        
        if not answer_text:  # 답변이 없는 경우도 처리
            answer_text = [""]

        for answer in answer_text:
            data.append([title, question_txt, date, answer])

df = pd.DataFrame(data, columns=['제목', '질문', '작성일', '답변'])
print(df)

# 필요에 따라 데이터를 파일로 저장
df.to_excel('naver_kin_crawling_result.xlsx', index=False)

driver.quit()


                             제목  \
0   질문\n강아지 놀아준게 잘못 된 행동을 한건가요?   
1                    질문\n강아지 설사   
2                    질문\n강아지 설사   
3                질문\n강아지 배 두드러기   
4                질문\n강아지 배 두드러기   
5                  질문\n강아지 새콤달콤   
6                  질문\n강아지 책...   
7                    질문\n강아지 비명   
8                    질문\n강아지 비명   
9                    질문\n강아지 비명   
10                   질문\n강아지 비명   
11            질문\n강아지 초콜릿 먹고 혈뇨   
12            질문\n강아지 초콜릿 먹고 혈뇨   
13            질문\n강아지 초콜릿 먹고 혈뇨   
14            질문\n강아지 초콜릿 먹고 혈뇨   
15            질문\n강아지 초콜릿 먹고 혈뇨   
16            질문\n강아지 초콜릿 먹고 혈뇨   

                                                   질문    작성일  \
0   본인 사업장(운동센터)이라 강아지 데리고 오시는 사장님들은 다른 직원이나 고객이 강...  NAVER   
1   저희집 강아지가 이번에 귀 문제 때문에 약을 먹기 시작 했는데 그때 부터 설사를 하...  NAVER   
2   저희집 강아지가 이번에 귀 문제 때문에 약을 먹기 시작 했는데 그때 부터 설사를 하...  NAVER   
3   주변에 24시 병원이없어서 지금 당장\n동물병원을 못가는 상황인데 내일가도 괜찮을까...  NAVER   
4   주변에 24시 병원이없어서 지금 당장\n동물병원을 못가는 상황인데 내일가도 괜찮을

In [28]:
df

,제목,질문,작성일,답변
0,질문\n강아지 놀아준게 잘못 된 행동을 한건가요?,본인 사업장(운동센터)이라 강아지 데리고 오시는 사장님들은 다른 직원이나 고객이 강...,NAVER,본인 사업장(운동센터)이라 강아지 데리고 오시는 사장님들은 다른 직원이나 고객이 강...
1,질문\n강아지 설사,저희집 강아지가 이번에 귀 문제 때문에 약을 먹기 시작 했는데 그때 부터 설사를 하...,NAVER,약을 어떤거 먹이고 계실까요\n저희개도 귓병 피부병 땜에 항생제 먹은적 있는데 항생...
2,질문\n강아지 설사,저희집 강아지가 이번에 귀 문제 때문에 약을 먹기 시작 했는데 그때 부터 설사를 하...,NAVER,저희집 강아지도 약 성분이 안맞을때 구토나 설사 하곤 해요! 병원가서 약 바꿔보시고...
3,질문\n강아지 배 두드러기,주변에 24시 병원이없어서 지금 당장\n동물병원을 못가는 상황인데 내일가도 괜찮을까...,NAVER,괜찮습니다 ㅎㅎ
4,질문\n강아지 배 두드러기,주변에 24시 병원이없어서 지금 당장\n동물병원을 못가는 상황인데 내일가도 괜찮을까...,NAVER,안녕하세요.\n지식iN 동물의료상담 수의사 새은평동물의료센터장 버드나무 이상철입니다...
5,질문\n강아지 새콤달콤,강아지가 종이 포장 된 새콤달콤 2개 먹었는데 괜찮나요? 맛은 블루베리 소다맛인데 ...,NAVER,강아지가 종이 포장 된 새콤달콤 2개 먹었는데 괜찮나요? 맛은 블루베리 소다맛인데 ...
6,질문\n강아지 책...,강아지 책 찾아요\n소설이고 시리즈로 3편 정도 있던걸로 기억해요\n하나는 앞다리가...,NAVER,혹시 보는 사람마다 윙크하는 윙키 아닌가요 저도 본 지 오래되서 아닐수도 잇어요ㅜ
7,질문\n강아지 비명,강아지가 오늘 저녁부터 갑자기 비명을 질러요.\n지금까지 두 세번 정도 연속적으로 ...,NAVER,안녕하세요 수의사 김원장입니다.\n.\n반려동물이 비명을 지르는 경우\n가장 흔한 ...
8,질문\n강아지 비명,강아지가 오늘 저녁부터 갑자기 비명을 질러요.\n지금까지 두 세번 정도 연속적으로 ...,NAVER,
9,질문\n강아지 비명,강아지가 오늘 저녁부터 갑자기 비명을 질러요.\n지금까지 두 세번 정도 연속적으로 ...,NAVER,
